In [2]:
from IPython.display import clear_output
!pip install datasets
clear_output()

In [22]:
from datasets import load_dataset

l_task1 = load_dataset("world-cuisines/vqa", "task1", split = "test_large").to_pandas()
l_task2 = load_dataset("world-cuisines/vqa", "task2", split = "test_large").to_pandas()

s_task1 = load_dataset("world-cuisines/vqa", "task1", split = "test_small").to_pandas()
s_task2 = load_dataset("world-cuisines/vqa", "task2", split = "test_small").to_pandas()

In [23]:
import pandas as pd

merged1 = pd.merge(
    l_task1, 
    s_task1, 
    on=['lang', 'prompt_id', 'prompt_type', 'food_id'],
    how='inner',
    suffixes=('_large', '_small')
)

merged2 = pd.merge(
    l_task2, 
    s_task2, 
    on=['lang', 'prompt_id', 'prompt_type', 'food_id'],
    how='inner',
    suffixes=('_large', '_small')
)


qa_id_matches1 = merged1['qa_id_large'].values.tolist()
qa_id_matches2 = merged2['qa_id_large'].values.tolist()

assert len(qa_id_matches1) == 9000
assert len(qa_id_matches2) == 3000

small_id_set = set(qa_id_matches2 + qa_id_matches1)
assert len(small_id_set) == 400

In [34]:
import os
import json
from tqdm import tqdm

input_dir = '../evaluation/result/'
output_dir = '../evaluation/result/small/'

os.makedirs(output_dir, exist_ok=True)

total_instances = 0
model_set = set()

for filename in tqdm(os.listdir(input_dir)):
    input_filepath = os.path.join(input_dir, filename)

    # Skip directories, only process files
    if os.path.isfile(input_filepath) and filename.endswith('.jsonl'):
        model_set.add(filename.split("_")[2])
        filtered_instances = []

        with open(input_filepath, 'r', encoding='utf-8') as infile:
            for line in infile:
                instance = json.loads(line)
                qa_id = instance.get('qa_id')

                if qa_id in small_id_set or str(qa_id) in small_id_set:
                    filtered_instances.append(instance)

        output_filepath = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_small.jsonl")
        with open(output_filepath, 'w', encoding='utf-8') as outfile:
            for instance in filtered_instances:
                outfile.write(json.dumps(instance) + '\n')
        
        assert len(filtered_instances) == 9_000 if "task1" in filename else 3_000


100%|██████████| 70/70 [00:06<00:00, 10.13it/s]
